# Collecting Textual News Data from Twitter


## Importing Dependencies

In [ ]:
from dotenv import load_dotenv
import os
import tweepy
import pandas as pd

## Setting Up Twitter API and Tokens from .env File

In [ ]:
load_dotenv()

## Setting Up Twitter Screaper

In [ ]:
tweets = []
userID = []
tweetID = []
tweet_url = []
username = []
next_page_token = []

In [ ]:
def getClient():
    
    client = tweepy.Client(bearer_token= os.getenv("BEARER_TOKEN"),
                           consumer_key=os.getenv("API_KEY"), 
                           consumer_secret=os.getenv("API_KEY_SECRET"), 
                           access_token=os.getenv("ACCESS_TOKEN"),
                           access_token_secret=os.getenv("ACCESS_TOKEN_SECRET"))

    return client


def searchTweets(query, qty):
    
    client = getClient()
    client_call = client.search_all_tweets(query=query, max_results=qty, expansions='author_id')
    
    if not client_call.data is None and len(client_call.data) > 0: #Checking if there are results or not before looping...
        for tweet in client_call.data:
            tweets.append(tweet.text)
            id_ = tweet.id
            tweetID.append(id_)
            user_id = tweet.author_id
            userID.append(user_id)
            tweet_url.append('https://twitter.com/{}/status/{}'.format(user_id, id_))
            
    else:
        return []
    
    
    try:
        next_page = client_call.meta['next_token']
        next_page_token.append(next_page)
    except KeyError:
        print("NO NEXT TOKEN AVAILABLE, THIS MEANS YOU GOT ONLY ONE PAGE.... :( ")

    df = pd.DataFrame({'Tweets': tweets, "Tweet URL": tweet_url, 'Tweet ID': tweetID, "User ID": userID})
    # df = df[~df.Tweets.str.contains("RT")]
           
    return df 


def search_tweets_next(query, qty, next_page_token, pagination):
    
    client = getClient()

    client_call_ = client.search_all_tweets(query=query, max_results=qty, expansions='author_id', next_token=next_page_token[pagination])
    #If you want to keep collecting, change to 20 the list above
    if not client_call_.data is None and len(client_call_.data) > 0: #Checking if there are results or not before looping...
        for tweet in client_call_.data:
            tweets.append(tweet.text)
            id_ = tweet.id
            tweetID.append(id_)
            user_id = tweet.author_id
            userID.append(user_id)   
            tweet_url.append('https://twitter.com/{}/status/{}'.format(user_id, id_))
    else:
        return []
    
    try:
        next_page = client_call.meta['next_token']
        next_page_token.append(next_page)
    except KeyError:
        print("NO NEXT TOKEN AVAILABLE, THIS MEANS YOU GOT ONLY ONE PAGE.... :( ")
    
    df = pd.DataFrame({'Tweets': tweets, "Tweet URL": tweet_url, 'Tweet ID': tweetID, "User ID": userID})
        
    return df

# Calling Function to search for tweets
This function will get all tweets in a dataframe and then it will be exported on a CSV file

In [ ]:
#First Data Request
df_1 = searchTweets('from:BBCWorld covid-19 -is:retweet lang:en', 500)    


In [ ]:
#Data requests adding pagination token (each request gets 500 tweets, every single execuion I have to change the token manually)
pagination = 0
df_ = search_tweets_next('covid-19 news -is:retweet lang:en', 500, next_page_token, pagination)




In [ ]:
frames = [df_1, df_2, df_3]

df = pd.concat(frames)
df

In [ ]:
userID

In [ ]:
# df.to_csv('Collected_Data/COVID-19.csv')

In [ ]:
len(next_page_token)